In [1]:
import json
import pandas as pd
import importlib
import pickle

In [2]:
filepath = '../scraping/articles/filtered_articles.json'
with open(filepath, 'r') as file:
    articles = pd.read_json(filepath)

filepath = '../relation_extraction/directory.pkl'

with open(filepath, 'rb') as file:
    keywords = pickle.load(file)

In [105]:
from relik import Relik
from relik.inference.data.objects import RelikOutput

relik = Relik.from_pretrained("sapienzanlp/relik-entity-linking-large")
relik_out: RelikOutput = relik("Michael Jordan was one of the best players in the NBA.")

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/Users/tiril/Documents/IndividualProject/nuclear_repo/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.yaml:   0%|          | 0.00/637 [00:00<?, ?B/s]

[2024-08-06 13:08:02,193] [INFO] [relik.inference.annotator.from_pretrained:694] [PID:29125] [RANK:0] Loading Relik from sapienzanlp/relik-entity-linking-large
[2024-08-06 13:08:02,195] [INFO] [relik.inference.annotator.from_pretrained:695] [PID:29125] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'span': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-e5-base-v2-aida-blink-wikipedia-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.span.RelikReaderForSpanExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-large-aida',
        'use_nme': True,
    },
    'retriever': {
        'span': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'sapienzanlp/relik-retriever-e5-base-v2-aida-bli

config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/197 [00:00<?, ?B/s]

documents.jsonl:   0%|          | 0.00/3.03G [00:00<?, ?B/s]

embeddings.pt:   0%|          | 0.00/9.07G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [78]:
import KnowledgeExtractor
importlib.reload(KnowledgeExtractor)

extractor = KnowledgeExtractor.KnowledgeExtractor('dslim/bert-base-NER', keywords)

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/Users/tiril/Documents/IndividualProject/nuclear_repo/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-08-06 11:51:07,074] [INFO] [relik.inference.annotator.from_pretrained:694] [PID:29125] [RANK:0] Loading Relik from sapienzanlp/relik-relation-extraction-nyt-large
[2024-08-06 11:51:07,075] [INFO] [relik.inference.annotator.from_pretrained:695] [PID:29125] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-small-nyt-document-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-large-nyt',
    },
    'retriever': {
        'triplet': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'sapienzanlp/relik-retriever-small-nyt-question-encoder',
        

In [94]:
instance = articles.iloc[57]
sents = instance['text']
print(sents)
#triplets = extractor.extract_relik_triplets(sents)


['Unit was shut down for a period following detachment of reactor support', 'Argentina’s nuclear regulator has issued a renewed licence to state operator Nucleoelectrica Argentina for the operation of the Atucha-2 nuclear power plant until 26 May 2026.', 'The Autoridad Regulatoria Nuclear (ARN) said the new licence would take Atucha-2, a 693-MW pressurised heavy water reactor (PHWR) unit that began commercial operation in May 2016, up to the end of its first 10 years of operation, a decade which has seen a number of licence extensions.', 'In December 2015 ARN granted a conditional operating licence for Atucha-2, about 100 km northwest of Buenos Aires, until May 2016. It issued its initial five-year licence on 26 May 2016 following the completion of a programme of testing and training. ', 'The first, two-year extension was granted in May 2021, but the unit was shut down from October 2022 for repairs after the discovery that one of the four internal supports of the reactor had detached.'

In [104]:
triplets = []
sent = sents[2]
print(len(sent))
print(sent)
out: RelikOutput = relik(sent[:200])
print(out)


285
The Autoridad Regulatoria Nuclear (ARN) said the new licence would take Atucha-2, a 693-MW pressurised heavy water reactor (PHWR) unit that began commercial operation in May 2016, up to the end of its first 10 years of operation, a decade which has seen a number of licence extensions.
[2024-08-06 13:06:16,993] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 1 number of elements processed


IndexError: list index out of range

In [90]:
output = []
for i in range(len(articles)):
    instance = articles.iloc[i]
    url = instance['url']
    sents = instance['text']
    entities = extractor.extract_entities(sents)
    triplets = extractor.extract_relik_triplets(sents)
    output.append({
        'url': url,
        'triplets': triplets,
        'entities': entities
    })

print(len(output))

with open('entities_and_triplets.json', 'w') as f:
    json.dump(output, f, indent=4)

[2024-08-06 12:08:13,393] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 20 number of elements processed
[2024-08-06 12:08:34,782] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 16 number of elements processed
[2024-08-06 12:08:55,249] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 15 number of elements processed
[2024-08-06 12:09:17,141] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 22 number of elements processed
[2024-08-06 12:09:39,886] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 25 number of elements processed
[2024-08-06 12:10:01,581] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 18 number of elements processed
[2024-08-06 12:10:23,777] [DEBUG] [relik.reade

IndexError: list index out of range

In [88]:
for o in output:
    print(o['url'])
    print(o['triplets'])
    print(o['entities'])

https://www.nucnet.org/news/kyrgyzstan-6-5-2024
[{'head': 'Kul Province', 'relation': 'contains', 'tail': 'Issyk', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}, {'head': 'EBRD', 'relation': 'company', 'tail': 'European Bank for Reconstruction and Development', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Central Asia', 'confidence': 0.9100000262260437}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Kyrgyzstan', 'confidence': 1.0}, {'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}]
[{'word': 'Bishkek', 'entity': 'ORG'}, {'word': 'Kyrgyzstan', 'entity': 'LOC'}, {'word': 'Kyrgyz', 'entity': 'LOC'}, {'word': 'nuclear power plant', 'entity': 'REACTOR'}, {'word': 'Sadyr Japarov', 'entity': 'PER'}, {'word': 'Issyk-Kul Province', 'entity': 'LOC'}, {'word': 'Kyzyl-Ompol', 'entity': 'LOC'}, {'word': 'Japarov', 'entity': 'PER'}, {'word': 'Kyrgyzstan', 'entity': 'LOC'}, {'word'

In [67]:
def extract_relik_triplets(sents):
    extracted_triplets = []

    relik_out: RelikOutput = relik(sents)

    for output in relik_out:
        output_dict = asdict(output)
        triplets = output_dict['triplets']
        if triplets:
            for triplet in triplets:
                head = triplet.subject.text
                tail = triplet.object.text
                label = triplet.label
                confidence = triplet.confidence
                extracted_triplets.append({
                    'head': head,
                    'relation': label,
                    'tail': tail,
                    'confidence': confidence
                }) 
    
    return extracted_triplets

In [68]:
sents = articles.iloc[0]['text']
triplets = extract_relik_triplets(sents)

[2024-08-06 11:28:04,415] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 20 number of elements processed


In [69]:
for t in triplets:
    print(t)

{'head': 'Kul Province', 'relation': 'contains', 'tail': 'Issyk', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}
{'head': 'EBRD', 'relation': 'company', 'tail': 'European Bank for Reconstruction and Development', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Central Asia', 'confidence': 0.9100000262260437}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Kyrgyzstan', 'confidence': 1.0}
{'head': 'Asia', 'relation': 'contains', 'tail': 'Soviet Union', 'confidence': 1.0}
